In [1]:
import openpyxl
from openpyxl import Workbook,load_workbook
from openpyxl.utils import get_column_letter
from datetime import date 
currentday=(date.today().strftime("%Y%m%d"))

## Step.1 - LOAD XML DATA IN OPENPYXL and Archive process

##### Step1.1 Loading required xl data

In [2]:
cs = load_workbook('Checking_Schedule.xlsx')  #loading cs data 
ws_cs=cs.active

tevon = load_workbook('TEVON_DOWNLOAD.xlsx')  #loading tevon data
ws_tevon= tevon.active

cs_backup = load_workbook('Checking_Schedule_backup.xlsx')  #Loading archive data for synchronization
ws_cs_backup= cs_backup.active

pr_f0001 = load_workbook('F0001.xlsx')  #loading tevon data
ws_f0001= pr_f0001.active

pr_f0002 = load_workbook('F0002.xlsx')  #loading tevon data
ws_f0002= pr_f0002.active


##### Step1.2 Archiving a copy 

In [3]:
cs.save('Checking_Schedule_{}.xlsx'.format(currentday))  #archiving the copy

## Step .2 DEFINE PARAMTERS

##### 2.1 Parameters define the row and col values

In [4]:

row_tevon = ws_tevon.max_row #max row in tevon
col_tevon = ws_tevon.max_column #max col in cs

row_cs = ws_cs.max_row  #max row in cs
row_cs_bu = ws_cs_backup.max_row 
col_cs = ws_cs.max_column #max col in cs

ls_tev=[2,3,4,5,6] # list of columns need to copy from tevon
ls_cs=[1,2,3,4,9]   # list of columns need to copy from cs

col_partnumber_cs=2 # columns with the part numbers in checking schedule
col_partnumber_cs_bu=2 # columns with the part numbers in checking schedule_backup

pr_f001_col =2 #column in which the  pr codes are present IN #F0001 excel file
pr_f002_col =2 #column in which the  pr codes are present IN #F0002 excel file

row_f0001 = ws_f0001.max_row #max row in f0001 excel
row_f0002 = ws_f0001.max_row #max row in f0002 excel

col_pr=14 #column in CS with pr code present
col_pr_comment=13 #column in CS with pr code comment

r_status = 5 # row with part check is  relevent or not, NA/N/Y,comment present


##### 2.2 PR code comments

In [5]:
comment1 = 'CODE PRESENT IN 01 FILTER'
comment2 = 'CODE PRESENT IN 02 FILTER' 
comment3 ='CODE NOT PRESENT IN 01 FILTER ({})'
comment4 ='CODE NOT PRESENT IN 01 FILTER ({})'
comment5 ='CODE NOT PRESENT IN 01/02 FILTER ({})'
comment6='CODE NOT PRESENT IN 01 FILTER ({},{})'
comment7='CODE NOT PRESENT IN 02 FILTER ({},{})'

## Step.3 - COPY DATA FROM TEVON TO CHECKING SCHEDULE

In [6]:
#copy to first section from tevon to CS
for r in range (1, row_tevon + 1):
    for c in range (2,6):
        ws_cs.cell(row=r,column=c).value = ws_tevon.cell(row=r,column=c).value 
               
#copy the second section from tevon  to CS
for r in range (1, row_tevon + 1):
    for c_tev,c_cs in zip(ls_tev,ls_cs):
        ws_cs.cell(row=r,column=c_cs).value = ws_tevon.cell(row=r,column=c_tev).value

In [6]:
def copy_data(start_row,last_row)
    for r in range (start_row, last_row + 1):
        for c in list_col:
            ws_cs.cell(row=r,column=c).value = ws_tevon.cell(row=r,column=c).value 
# input are as folows
# 1.start row and end row
# 3.list of columns to copy in list format

In [7]:
cs.save('CS_after_Tevon_copy_{}.xlsx'.format(currentday))  #SAVING  a copy after tevon copy

## Step.4 REMOVE SPACE FROM PART NUMBERS

In [8]:
def remove_space(work_sheet,first_row,total_rows,col_to_change):
    for r in range (first_row, total_rows + 1):
            val=(str(work_sheet.cell(row=r,column=col_to_change).value)).replace(" ","") #str command converts if the value is an int
            work_sheet.cell(row=r,column=col_to_change).value =val  #change this to int from float on original cs


In [27]:
work_sheet =ws_cs
first_row =2
total_rows =row_cs
col_to_change=col_partnumber_cs

work_sheet=remove_space(work_sheet,first_row,total_rows,col_to_change)

ws_cs=work_sheet

AttributeError: 'NoneType' object has no attribute 'cell'

In [28]:
cs.save('CS_space_removed.xlsx'.format(currentday))  #SAVING  a copy after space removed

In [11]:
# for r in range (2, row_cs + 1):
#         val=(str(ws_cs.cell(row=r,column=col_partnumber_cs).value)).replace(" ","") #str command converts if the value is an int
#         ws_cs.cell(row=r,column=col_partnumber_cs).value =val  #change this to int from float on original cs

## Step.5  COPY THE BACKUP TO THE NEW CS LOOKUP

In [13]:
cs_copy=[3,4,5,6,7]  #coluims where need to copy to data from cs_bakcup
cs_backup_copy=[8,9,10,11,12] # data colums we need to copy from backup to new cs

for r_cs in range(2,row_cs + 1):
    for r_cs_bu in range(2,row_cs_bu + 1):
        if ws_cs.cell(row=r_cs,column=col_partnumber_cs).value == ws_cs_backup.cell(row=r_cs_bu,column=col_partnumber_cs_bu).value :
            for col1,col2 in zip(cs_copy,cs_backup_copy):
                ws_cs.cell(row=r_cs,column=col1).value = ws_cs_backup.cell(row=r_cs_bu,column=col2).value

In [14]:
cs.save('CS_after_sync_{}.xlsx'.format(currentday))  #SAVING  a copy after Sync

## Step.6  Partial checkup

#####  6.1 Identify the row numbers of newly added parts in a list format

In [15]:
new_added_part=[] # list to store the newly added part's row numbers
for r_pr_comm in range(1,row_cs+1):
    if ws_cs.cell(row=r_pr_comm,column=r_status).value== None:
        new_added_part.append(r_pr_comm)

###### 6.2 FILL COMMON PART COMMENT IF PR CODE NOT PRESENT

In [16]:
for r_col_partial in new_added_part:
    pr=ws_cs.cell(row=r_col_partial,column=col_pr).value
    if pr == None:
        ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = 'PR CODE NOT PRESENT COMMON PART'

In [17]:
cs.save('Checking_Schedule.xlsx')  #SAVING  final excel

##### 6.3 TO CREATE THE LIST OF PR CODES IN F0001 and F0002

In [18]:
# TO CREATE THE LIST OF PR CODES IN F0001
LOL=['']
for r_f0001 in range(1,row_f0001+1):   #index all the VALUES IN 8TH COLUMN ie.LOL
        LOL.append(ws_f0001.cell(row=r_f0001,column=pr_f001_col).value)

# TO CREATE THE LIST OF PR CODES IN F0002
LOR=['']
for r_f0002 in range(1,row_f0002+1):   #index all the VALUES IN 8TH COLUMN ie.LOL
        LOR.append(ws_f0002.cell(row=r_f0002,column=pr_f002_col).value)

###### 6.4   RUN BELOW CODE TO GET LIST WITH EMPTY PR_COMMENT TO OMIT THE ROWs  FILLED WITH comment "PR CODE NOTPRESENT COMMON PART"

In [19]:
row_no_prcomment=[] # list to store the newly added part's row numbers
for r_pr_comm in range(1,row_cs+1):
    if ws_cs.cell(row=r_pr_comm,column=col_pr_comment).value== None:
        row_no_prcomment.append(r_pr_comm)

###### 6.5 START TO ADDRESS THE REMAINING ROWS WITH PR CODES

In [20]:
for r_col_partial in row_no_prcomment:
    pr=ws_cs.cell(row=r_col_partial,column=col_pr).value #index all the newly added rows prcode value
    if '/' not in pr: #handling  the pr without 'OR' conditiion in it
        if 'L0R' in pr: #HANDLE PR CODE WITH l0R
            if all(x in LOR for x in pr.split('+')) :
                ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment2
                #entering comment on pr comment cells
            else:
                for x in pr.split('+'):
                    if x not in LOR:
                        missingLOR=x
                        ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment4.format(missingLOR)
                        
            
        elif 'L0L' in pr: # HANDLE PR CODE WITH l0l
            if all(x in LOL for x in pr.split('+')) :
                ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment1 
                #entering comment on pr comment cells
                
            else:
                for x in pr.split('+'):
                    if x not in LOL:
                        missingLOL=x
                        ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment3.format(missingLOL)
        # HANDLE REMAINING PR CODES WITHOUT LOR , LOR and '/'   
        elif all(x in LOL for x in pr.split('+')) :
            ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment1 
            #entering comment on pr comment cells
        elif all(x in LOR for x in pr.split('+')) :
            ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment2
            #entering comment on pr comment cells
        else:
            for x in pr.split('+'):
                if x not in LOL:
                    missingLOL=x
                    ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment5.format(missingLOL)
                    break
   #Handle pr code with '/' OR        
    else:
        #print(pr, 'with/')
        listofslashgroup=[]  #list to store the group with slash(OR ,'/'') ,ie  A1/A2/D5/C8
        listwithoutslash=[]  #list to store the group without slash,ie indivitual seperated with + ,A1+A2+B5
        slashinLOL=0
        slashinLOR=0
        pr_with_or=pr.split('+')
        for v in pr_with_or:
            if '/' in v:
                listofslashgroup.append(v) # added all the pr codes with OR to one list
            else:
                listwithoutslash.append(v) # added all the pr codes without OR to one list
# first handle all the +pr codes in the group of OR
        allplusinlol=False
        allplusinlor=False
        slashinLOL=False
        slashinLOR=False            
        if all(x in LOL for x in listwithoutslash) :
            allplusinlol=True
        else:#below to store a missing value from +group
            for x in listwithoutslash:
                if x not in LOL:
                    LOLmissing=x
            
        if all(x in LOR for x in listwithoutslash) :
            allplusinlor=True
        else:
            for x in listwithoutslash:
                if x not in LOR:
                    LORmissing=x                          
#below else codes is to set a flag if all the '/' groups are meeting or not the LOL and LOR
        for v1 in listofslashgroup: 
            if any(x in LOL for x in v1.split('/')):
                slashinLOL=True
                continue
            else:
                LOLmissingwithinslash = v1  #store one value from the slash as an input to the comment

        for v2 in listofslashgroup: 
            if any(x in LOR for x in v2.split('/')):
                slashinLOR=True
                continue
            else:
                LORmissingwithinslash = v2  #store one value from the slash as an input to the comment 
                
# checking the flag to decide the outcome

        if  allplusinlol and slashinLOL:
            ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment1  
        elif  allplusinlor and slashinLOR:
            ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment2
        elif  allplusinlol == False and slashinLOL:
             ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment3.format(LOLmissing)
        elif  allplusinlol == True and slashinLOL==False:
             ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment3.format(LOLmissingwithinslash)
        elif  allplusinlor == False and slashinLOR:
             ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment4.format(LORmissing)
        elif  allplusinlor == True and slashinLOL==False:
             ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment3.format(LOLmissingwithinslash)
        elif  allplusinlol == False and slashinLOL==False:
             ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value =comment6.format(LOLmissing,LOLmissingwithinslash)
        elif  allplusinlor == False and slashinLOR==False:
             ws_cs.cell(row=r_col_partial ,column=col_pr_comment).value = comment7.format(LORmissing,LORmissingwithinslash)
            

## Step.7  Partial checkup

In [21]:
cs.save('Checking_Schedule_final.xlsx')  #SAVING  final excel